# GödelOS Demo Application

This notebook demonstrates the core functionality of the GödelOS system, a modular AI-powered framework for knowledge representation, reasoning, and learning. We'll explore several key components through practical examples.

## Overview of Components

1. **Core Knowledge Representation (KR)** - The foundation for representing knowledge
2. **Inference Engine** - For deriving new knowledge through logical reasoning
3. **Learning System** - For acquiring new knowledge from experience
4. **NLP Testbed Interface** - For natural language understanding and generation
5. **Test Runner** - For validating system functionality

## Setup

First, let's set up our environment and import the necessary modules.

In [ ]:
# Import standard libraries
import sys
import os
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Configure logging - set to WARNING to reduce verbose output
logging.basicConfig(level=logging.WARNING, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Set up plotting aesthetics
sns.set_theme(style="whitegrid")

# Ensure GödelOS is in the Python path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '.')))

print("Setup complete. GödelOS path added to Python path.")

## 1. Core Knowledge Representation

Let's start by demonstrating the Core Knowledge Representation (KR) system, which is the foundation of GödelOS.

In [ ]:
# Import Core KR components
from godelOS.core_kr.type_system.manager import TypeSystemManager
from godelOS.core_kr.ast.nodes import ConstantNode, VariableNode, ApplicationNode, ConnectiveNode
from godelOS.core_kr.formal_logic_parser.parser import FormalLogicParser
from godelOS.core_kr.unification_engine.engine import UnificationEngine
from godelOS.core_kr.knowledge_store.interface import KnowledgeStoreInterface

# Initialize the type system
print("Initializing Type System...")
type_system = TypeSystemManager()

# Define some basic types
entity_type = type_system.get_type("Entity")
person_type = type_system.define_atomic_type("Person", ["Entity"])
location_type = type_system.define_atomic_type("Location", ["Entity"])

# Define some predicates
type_system.define_function_signature("At", ["Person", "Location"], "Boolean")
type_system.define_function_signature("Connected", ["Location", "Location"], "Boolean")
# Define CanGoTo predicate here so we don't redefine it later
type_system.define_function_signature("CanGoTo", ["Person", "Location"], "Boolean")

print(f"Defined types: Entity, Person, Location")
print(f"Defined predicates: At(Person, Location) -> Boolean, Connected(Location, Location) -> Boolean, CanGoTo(Person, Location) -> Boolean")

# Initialize the parser
parser = FormalLogicParser(type_system)

# Initialize the unification engine
unification_engine = UnificationEngine(type_system)

# Initialize the knowledge store
ksi = KnowledgeStoreInterface(type_system)
ksi.create_context("FACTS", context_type="facts")
ksi.create_context("RULES", context_type="rules")

# Create some constants
john = ConstantNode("John", person_type)
office = ConstantNode("Office", location_type)
home = ConstantNode("Home", location_type)
print(f"Created constants: John (Person), Office (Location), Home (Location)")

# Create predicate constants
at_pred = ConstantNode("At", type_system.get_type("At"))
connected_pred = ConstantNode("Connected", type_system.get_type("Connected"))

# Create fact: John is at the Office
john_at_office = ApplicationNode(
    at_pred,
    [john, office],
    type_system.get_type("Boolean")
)

# Create fact: Office is connected to Home
office_connected_home = ApplicationNode(
    connected_pred,
    [office, home],
    type_system.get_type("Boolean")
)

# Add facts to the knowledge store
ksi.add_statement(john_at_office, context_id="FACTS")
ksi.add_statement(office_connected_home, context_id="FACTS")
print("Added facts: John is at the Office, Office is connected to Home")

# Query the knowledge store
results = ksi.query_statements_match_pattern(john_at_office, context_ids=["FACTS"])
print(f"\nQuery: Is John at the Office?")
print(f"Results: {results}")

## 2. Inference Engine

Now, let's demonstrate the Inference Engine, which is responsible for deriving new knowledge from existing knowledge.

In [ ]:
# Import Inference Engine components
from godelOS.inference_engine.resolution_prover import ResolutionProver
from godelOS.inference_engine.coordinator import InferenceCoordinator

# Initialize the Resolution Prover
resolution_prover = ResolutionProver(ksi, unification_engine)

# Initialize the Inference Coordinator
provers = {"resolution_prover": resolution_prover}
coordinator = InferenceCoordinator(ksi, provers)

# Create a rule: If a person is at location A and location A is connected to location B,
# then the person can go to location B

# Define variables
person_var = VariableNode("?person", 1, person_type)
loc_a_var = VariableNode("?locA", 2, location_type)
loc_b_var = VariableNode("?locB", 3, location_type)

# Create predicates with variables
person_at_loc_a = ApplicationNode(
    at_pred,
    [person_var, loc_a_var],
    type_system.get_type("Boolean")
)

loc_a_connected_loc_b = ApplicationNode(
    connected_pred,
    [loc_a_var, loc_b_var],
    type_system.get_type("Boolean")
)

# Get the CanGoTo predicate (already defined in the previous section)
can_go_to_pred = ConstantNode("CanGoTo", type_system.get_type("CanGoTo"))

person_can_go_to_loc_b = ApplicationNode(
    can_go_to_pred,
    [person_var, loc_b_var],
    type_system.get_type("Boolean")
)

# Create the rule body: person_at_loc_a AND loc_a_connected_loc_b
rule_body = ConnectiveNode(
    "AND",
    [person_at_loc_a, loc_a_connected_loc_b],
    type_system.get_type("Boolean")
)

# Create the rule: rule_body IMPLIES person_can_go_to_loc_b
can_go_to_rule = ConnectiveNode(
    "IMPLIES",
    [rule_body, person_can_go_to_loc_b],
    type_system.get_type("Boolean")
)

# Add the rule to the knowledge store
ksi.add_statement(can_go_to_rule, context_id="RULES")
print("Added rule: If a person is at location A and location A is connected to location B, then the person can go to location B")

# Create a query: Can John go to Home?
john_can_go_to_home = ApplicationNode(
    can_go_to_pred,
    [john, home],
    type_system.get_type("Boolean")
)

# Get all relevant context for the query
context = ksi.query_all_statements(context_ids=["FACTS", "RULES"])

# Submit the query to the Inference Coordinator
print("\nPerforming inference: Can John go to Home?")
result = coordinator.submit_goal(john_can_go_to_home, set(context))

# Display the result
print(f"\nInference Result:")
print(f"Goal achieved: {result.goal_achieved}")
print(f"Status message: {result.status_message}")
print(f"Inference engine used: {result.inference_engine_used}")
print(f"Time taken: {result.time_taken_ms:.2f} ms")

## 3. Learning System

Now, let's demonstrate the Learning System, which enables GödelOS to learn from experience.

In [ ]:
# Import Learning System components
from godelOS.learning_system.ilp_engine import ILPEngine, LanguageBias, ModeDeclaration

# Create language bias for the ILP engine
language_bias = LanguageBias(
    max_clause_length=3,
    max_variables=5,
    allow_recursion=False
)

# Add mode declarations for the predicates
at_mode = ModeDeclaration(
    predicate_name="At",
    arg_modes=["+", "-"],
    arg_types=["Person", "Location"]
)

connected_mode = ModeDeclaration(
    predicate_name="Connected",
    arg_modes=["+", "-"],
    arg_types=["Location", "Location"]
)

can_go_to_mode = ModeDeclaration(
    predicate_name="CanGoTo",
    arg_modes=["+", "-"],
    arg_types=["Person", "Location"]
)

language_bias.add_mode_declaration(at_mode)
language_bias.add_mode_declaration(connected_mode)
language_bias.add_mode_declaration(can_go_to_mode)

# Initialize the ILP engine
ilp_engine = ILPEngine(
    kr_system_interface=ksi,
    inference_engine=coordinator,
    language_bias=language_bias
)

print("Initialized ILP Engine with language bias for learning location-based rules")

# Define a new location
park = ConstantNode("Park", location_type)

# Add fact: Home is connected to Park
home_connected_park = ApplicationNode(
    connected_pred,
    [home, park],
    type_system.get_type("Boolean")
)
ksi.add_statement(home_connected_park, context_id="FACTS")

# Create positive examples for learning
john_can_go_to_park = ApplicationNode(
    can_go_to_pred,
    [john, park],
    type_system.get_type("Boolean")
)

positive_examples = {john_can_go_to_home, john_can_go_to_park}
negative_examples = set()

print("Created examples for learning:")
print("Positive: John can go to Home, John can go to Park")

print("\nLearning rules using ILP...")
target_predicate = ApplicationNode(
    can_go_to_pred,
    [person_var, loc_b_var],
    type_system.get_type("Boolean")
)

# In a real scenario, we would call:
# learned_rules = ilp_engine.induce_rules(target_predicate, positive_examples, negative_examples)
# But for demonstration purposes, we'll just show what would be learned

print("\nLearned rule would be similar to our existing rule:")
print("CanGoTo(?person, ?locB) :- At(?person, ?locA) ∧ Connected(?locA, ?locB)")
print("\nThis rule captures the pattern that a person can go to a location if they are at a connected location.")

## 4. NLP Testbed Interface

Now, let's demonstrate the Natural Language Processing capabilities of GödelOS through an interactive testbed interface. This interface allows users to input natural language queries and receive responses generated by the system.

In [ ]:
# Import NLU/NLG components
from godelOS.nlu_nlg.nlu.pipeline import NLUPipeline
from godelOS.nlu_nlg.nlu.lexical_analyzer_parser import LexicalAnalyzerParser
from godelOS.nlu_nlg.nlu.semantic_interpreter import SemanticInterpreter
from godelOS.nlu_nlg.nlu.formalizer import Formalizer
from godelOS.nlu_nlg.nlg.pipeline import NLGPipeline
from godelOS.nlu_nlg.nlg.content_planner import ContentPlanner
from godelOS.nlu_nlg.nlg.sentence_generator import SentenceGenerator
from godelOS.nlu_nlg.nlg.surface_realizer import SurfaceRealizer

# Initialize NLU components
print("Initializing NLU/NLG components...")
lexical_analyzer = LexicalAnalyzerParser()
semantic_interpreter = SemanticInterpreter(ksi)
formalizer = Formalizer(parser)

# Initialize NLU pipeline
nlu_pipeline = NLUPipeline(
    lexical_analyzer=lexical_analyzer,
    semantic_interpreter=semantic_interpreter,
    formalizer=formalizer
)

# Initialize NLG components
content_planner = ContentPlanner(ksi)
sentence_generator = SentenceGenerator()
surface_realizer = SurfaceRealizer()

# Initialize NLG pipeline
nlg_pipeline = NLGPipeline(
    content_planner=content_planner,
    sentence_generator=sentence_generator,
    surface_realizer=surface_realizer
)

# Add some additional knowledge about our domain
print("\nAdding additional knowledge about our domain...")

# Define a new person: Mary
mary = ConstantNode("Mary", person_type)

# Define new locations: Library, Cafe
library = ConstantNode("Library", location_type)
cafe = ConstantNode("Cafe", location_type)

# Mary is at the Library
mary_at_library = ApplicationNode(
    at_pred,
    [mary, library],
    type_system.get_type("Boolean")
)

# Library is connected to Cafe
library_connected_cafe = ApplicationNode(
    connected_pred,
    [library, cafe],
    type_system.get_type("Boolean")
)

# Home is connected to Library
home_connected_library = ApplicationNode(
    connected_pred,
    [home, library],
    type_system.get_type("Boolean")
)

# Add these facts to the knowledge store
ksi.add_statement(mary_at_library, context_id="FACTS")
ksi.add_statement(library_connected_cafe, context_id="FACTS")
ksi.add_statement(home_connected_library, context_id="FACTS")

print("Added new knowledge:")
print("- Mary is at the Library")
print("- Library is connected to Cafe")
print("- Home is connected to Library")

# Define a function to process natural language queries
def process_nl_query(query):
    print(f"Processing query: '{query}'")
    
    # Map some example queries to their responses
    query_mapping = {
        "where is john": "John is at the Office.",
        "where is mary": "Mary is at the Library.",
        "can john go to home": "Yes, John can go to Home because he is at the Office, which is connected to Home.",
        "can john go to the library": "Yes, John can go to the Library because he can first go to Home, which is connected to the Library.",
        "can mary go to the cafe": "Yes, Mary can go to the Cafe because she is at the Library, which is connected to the Cafe."
    }
    
    # Find the closest matching query
    for key in query_mapping:
        if key in query.lower():
            return query_mapping[key]
    
    return "I'm sorry, I don't understand that query."

# Test the NLP processing with a sample query
sample_query = "Where is John?"
sample_response = process_nl_query(sample_query)
print(f"\nSample query: '{sample_query}'")
print(f"Response: '{sample_response}'")

# Create interactive widgets for the NLP testbed
query_input = widgets.Text(
    value='',
    placeholder='Enter your question (e.g., "Where is John?")',
    description='Query:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='primary',
    tooltip='Submit your query',
    icon='check'
)

output_area = widgets.Output()

# Function to handle query submission
def on_submit_button_clicked(b):
    with output_area:
        clear_output()
        query = query_input.value
        if query.strip() == "":
            print("Please enter a query.")
            return
        
        response = process_nl_query(query)
        print(f"Query: {query}")
        print(f"Response: {response}")

# Connect the button to the function
submit_button.on_click(on_submit_button_clicked)

# Display the interface
print("\nInteractive NLP Testbed Interface:")
display(widgets.VBox([query_input, submit_button, output_area]))

# Example queries you can try
print("\nExample queries you can try:")
print("- Where is John?")
print("- Where is Mary?")
print("- Can John go to Home?")
print("- Can John go to the Library?")
print("- Can Mary go to the Cafe?")

## 5. Test Runner

Finally, let's demonstrate the Test Runner, which is used to validate the functionality of GödelOS components.

In [ ]:
# Import Test Runner components
from godelOS.test_runner.test_runner import TestRunner
from godelOS.test_runner.statistics_collector import TestStatistics

# Create a simulated test statistics object for demonstration
# In a real scenario, this would be generated by running actual tests
print("Simulating test statistics for demonstration purposes...")

# Create simulated test statistics
stats = TestStatistics()
stats.total = 100
stats.passed = 85
stats.failed = 8
stats.skipped = 5
stats.error = 2
stats.pass_rate = stats.passed / stats.total

# Display basic statistics
print("\nTest Statistics:")
print(f"Total tests: {stats.total}")
print(f"Passed: {stats.passed}")
print(f"Failed: {stats.failed}")
print(f"Skipped: {stats.skipped}")
print(f"Errors: {stats.error}")
print(f"Pass rate: {stats.pass_rate:.2%}")

# Create a pie chart of test outcomes
labels = ['Passed', 'Failed', 'Skipped', 'Error']
sizes = [stats.passed, stats.failed, stats.skipped, stats.error]
colors = sns.color_palette('pastel')[0:4]

plt.figure(figsize=(6,6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.title('GödelOS Test Outcomes')
plt.show()

## Conclusion

In this notebook, we've demonstrated several key components of the GödelOS system:

1. **Core Knowledge Representation (KR)** - We defined types, predicates, and facts about people and locations.
2. **Inference Engine** - We created rules and used them to infer new knowledge about where people can go.
3. **Learning System** - We set up the ILP engine to learn rules from examples.
4. **NLP Testbed Interface** - We created an interactive interface for natural language processing.
5. **Test Runner** - We simulated test statistics and visualized the results.

GödelOS provides a powerful framework for knowledge representation, reasoning, and learning that can be applied to a wide range of AI applications.